In [1]:
code = {
    "name": "commands",
    "body": [
        {
            "name": "repeat-n",
            "count": 10,
            "body": {
                "name": "commands",
                "body": [
                    {
                        "name": "gpio-write",
                        "pin": 16,
                        "value": 1
                    },
                    {
                        "name": "wait-sec",
                        "duration": 1
                    },
                    {
                        "name": "gpio-write",
                        "pin": 16,
                        "value": 0
                    },
                    {
                        "name": "wait-sec",
                        "duration": 1
                    }
                ]
            }
        }, {
            "name": "raw",
            "commands": [
                "ping 0",
                "ident"
            ]
        }
    ]
}

In [2]:
from proc_comp import parser

p = parser.parse(code)

#print(p)
print(p.pprint())

SeqExp([
  RepeatExp((U32) 10, [
  ])
  RawCSH([
    ping 0
    ident
  ])
])


In [3]:
from proc_comp.codegen.codegen import CodeGen

G = CodeGen()
G.code_gen(p)


['proc del 10 12',
 'proc new',
 'proc set _p_uint32[0] 0',
 'proc set _p_uint32[1] 10',
 'proc call 11',
 'ping 0',
 'ident',
 'proc push 10 12',
 'proc del 11 12',
 'proc new',
 'proc ifelse _p_uint32[0] < _p_uint32[1]',
 'proc call 12',
 'proc noop',
 'proc push 11 12',
 'proc del 12 12',
 'proc new',
 'proc set gpio_mode[16] 1',
 'proc set gpio_value[16] 1',
 'proc unop time idt _p_uint32[2]',
 'proc set _p_uint32[3] 1',
 'proc binop _p_uint32[2] + _p_uint32[3] _p_uint32[2]',
 'proc block time >= _p_uint32[2]',
 'proc set gpio_mode[16] 1',
 'proc set gpio_value[16] 0',
 'proc unop time idt _p_uint32[2]',
 'proc set _p_uint32[3] 1',
 'proc binop _p_uint32[2] + _p_uint32[3] _p_uint32[2]',
 'proc block time >= _p_uint32[2]',
 'proc unop _p_uint32[0] ++ _p_uint32[0]',
 'proc call 11',
 'proc push 12 12',
 'proc run 10 12']

In [4]:
parser.discover()

{'commands': {'fields': [{'name': 'body',
    'type': '[expression]',
    'default': None,
    'required': True,
    'attributes': {},
    'description': None}],
  'description': 'A list of expressions to be executed sequentially'},
 'if': {'fields': [{'name': 'cond',
    'type': 'conditional',
    'default': None,
    'required': True,
    'attributes': {},
    'description': None},
   {'name': 'then',
    'type': 'expression',
    'default': None,
    'required': True,
    'attributes': {},
    'description': None}],
  'description': 'Do something only if the condition is true'},
 'ifelse': {'fields': [{'name': 'cond',
    'type': 'conditional',
    'default': None,
    'required': True,
    'attributes': {},
    'description': None},
   {'name': 'then',
    'type': 'expression',
    'default': None,
    'required': True,
    'attributes': {},
    'description': None},
   {'name': 'else',
    'type': 'expression',
    'default': None,
    'required': True,
    'attributes': {},
    '